In [ ]:
!pip install scrapy boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.5/271.5 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 KB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 KB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 K

In [ ]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
path = 'drive/MyDrive/bloc1/' 

Mounted at /content/drive


In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# obtenir les coordonnées des villes


**Nous** allons récupérer les coordonnées des 35 premières villes grâce à l'API, en prenant les valeurs des temperatures moyennes sur les 7 prochains jours et les précipations minimales sue les 7 prochains jours 


In [ ]:
cities = [
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

list_latitude = []
list_longitude = []
list_temp_moyenne = []
list_pop_moyenne = []

for city in cities :
    # Recupere latitute et longitude de chaque ville
    r = requests.get(f"https://nominatim.openstreetmap.org/search?q={city}&format=json")
    premier_resultat=r.json()[0]
    lat= premier_resultat.get("lat")
    long =premier_resultat.get("lon")
    list_latitude.append(lat)
    list_longitude.append(long)
    
    # Récupere temperature moyenne et moyenne de probabilité de précipitation sur les sept ptochains jours 
    r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={long}&exclude=current,minutely,hourly&appid=6ebc4f03187866817713df5aae660c2f")
    resultat = r.json()
    temps = []
    pops=[]
    for day in range(0,7):
        temps.append(resultat.get('daily')[day].get('temp').get('day'))
        pops.append(resultat.get('daily')[day].get('pop'))
    
    list_temp_moyenne.append(np.mean(temps))
    list_pop_moyenne.append(np.mean(pops))


In [ ]:
data = pd.DataFrame()
data['city'] = cities
data['lat'] = list_latitude
data['long'] = list_longitude
data['temp'] = list_temp_moyenne
data['pop'] = list_pop_moyenne

In [ ]:
data = data.sort_values(by=['temp','pop'], ascending=[False, True])
data.to_csv(path+'results/cities.csv', index=True, index_label='id')

# Collecter les données des hotels


The goal is the collect to 20 best hotels from the 5 cities with the best weather in the next 7 days. It will be done using Scrapy on the Booking.com website

In [ ]:
# Lecture uniquement des 5 premières villes 
top5 = pd.read_csv(path +'results/cities.csv', nrows=5)
top5

,id,city,lat,long,temp,pop
0,26,Collioure,42.525050,3.083155,282.741429,0.335714
1,17,Bormes les Mimosas,43.150697,6.341928,281.827143,0.222857
2,18,Cassis,43.214036,5.539632,281.801429,0.224286
3,24,Aigues Mortes,43.565823,4.191284,281.754286,0.155714
4,19,Marseille,43.296174,5.369953,281.592857,0.234286


In [ ]:
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

In [ ]:
# List of urls the Spider will have the scrap
urls = [(row['id'], f'https://www.booking.com/searchresults.fr.html?ss={row["city"]}&order=score') for index, row in top5.iterrows()]

In [ ]:
urls

[(26,
  'https://www.booking.com/searchresults.fr.html?ss=Collioure&order=score'),
 (17,
  'https://www.booking.com/searchresults.fr.html?ss=Bormes les Mimosas&order=score'),
 (18, 'https://www.booking.com/searchresults.fr.html?ss=Cassis&order=score'),
 (24,
  'https://www.booking.com/searchresults.fr.html?ss=Aigues Mortes&order=score'),
 (19,
  'https://www.booking.com/searchresults.fr.html?ss=Marseille&order=score')]

# Les items choisis pour scrapper les données

In [ ]:
class HotelsSpider(scrapy.Spider):
    # Name of your spider
    name = "hotelsspider"
    

    
    def start_requests(self):
        for id, url in urls:
            yield scrapy.Request(url, meta={'id' : id})

    def parse(self, response):
        for result in response.css('a[data-testid="title-link"]')[:20]:
            yield scrapy.Request(result.attrib['href'], callback=self.parse_hotels, meta={'id' : response.meta.get('id')})
            
              
    def parse_hotels(self, response):  
        yield {
                'name': response.css('h2.pp-header__title::text').get(),
                'localisation': response.css('a#hotel_header').attrib['data-atlas-latlng'],
                'score' : response.css('div.b5cd09854e::text').get(),
                'description' : ''.join(response.css('div#property_description_content>p::text').getall()),
                'city_id' : response.meta.get('id')
            }

In [ ]:
# Name of the file where the results will be saved
filename = "hotels_top5.json"

# If file already exists, delete it before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir(path +'results/'):
        os.remove(path+'results/' + filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'LOG_LEVEL': logging.INFO,
    "AUTOTHROTTLE_ENABLED": True,
    "FEEDS": {
        path +'results/' + filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(HotelsSpider)
process.start()

INFO:scrapy.utils.log:Scrapy 2.7.1 started (bot: scrapybot)
2023-01-15 09:51:17 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.8.16 (default, Dec  7 2022, 01:12:13) - [GCC 7.5.0], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Linux-5.10.147+-x86_64-with-glibc2.27
2023-01-15 09:51:17 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.1.1, Twisted 22.10.0, Python 3.8.16 (default, Dec  7 2022, 01:12:13) - [GCC 7.5.0], pyOpenSSL 23.0.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 39.0.0, Platform Linux-5.10.147+-x86_64-with-glibc2.27
INFO:scrapy.crawler:Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36

# Création d'une carte pour le top 5

In [ ]:
fig = px.scatter_mapbox(
        top5, 
        lat="lat", 
        lon="long",
        mapbox_style="carto-positron",
        zoom=5,
        
)
fig.update_traces(marker={'size' : 10})

fig.show()

In [ ]:
top5

,id,city,lat,long,temp,pop
0,26,Collioure,42.525050,3.083155,282.741429,0.335714
1,17,Bormes les Mimosas,43.150697,6.341928,281.827143,0.222857
2,18,Cassis,43.214036,5.539632,281.801429,0.224286
3,24,Aigues Mortes,43.565823,4.191284,281.754286,0.155714
4,19,Marseille,43.296174,5.369953,281.592857,0.234286


In [ ]:
hotels = pd.read_json(path +'results/hotels_top5.json')
hotels

,name,localisation,score,description,city_id
0,Résidence Saint Vincent,"42.52730095,3.08423268","8,0",Vous pouvez bénéficier d'une réduction Genius ...,26
1,Studette idéalement située - Collioure,"42.52393500,3.08446940",None,"Situé à Collioure, à seulement 500 mètres du c...",26
2,T3 La Crique- Piscine -Terrasses -Wifi -Parking,"42.52713180,3.07614980",None,"Situé à Collioure, à seulement 1,6 km du châte...",26
3,Coconut - appartement T2 à Collioure,"42.52215520,3.08856820","8,0","Situated in Collioure, 300 metres from Boutigu...",26
4,Hôtel Princes de Catalogne,"42.52591431,3.08268160","7,9",Installé dans la ville pittoresque de Colliour...,26
...,...,...,...,...,...
95,Bel appartement pour 4 personnes - 4SAP22,"42.52434874,3.09014887","7,7",Vous pouvez bénéficier d'une réduction Genius ...,26
96,Le Mas des Citronniers,"42.52538159,3.08256391","8,0",Vous pouvez bénéficier d'une réduction Genius ...,26
97,La Frégate,"42.52638625,3.08325659","8,2","Situé à Collioure, l'établissement La Frégate ...",26
98,Appartement Les Roches Bleues - 5RB14,"42.52343100,3.09768800","9,0","Offrant une vue sur la mer, L'Appartement Les ...",26


In [ ]:
hotels[['lat','long']]=hotels.localisation.str.split(pat=',',expand=True)
hotels[['lat','long']] = hotels[['lat','long']].astype(float)
hotels['city_id'] = hotels['city_id'].astype(str)
top5['id']=top5['id'].astype(str)

In [ ]:
hotels = hotels.merge(top5, how='inner', left_on='city_id', right_on='id')
hotels

,name,localisation,score,description,city_id,lat_x,long_x,id,city,lat_y,long_y,temp,pop
0,Résidence Saint Vincent,"42.52730095,3.08423268","8,0",Vous pouvez bénéficier d'une réduction Genius ...,26,42.527301,3.084233,26,Collioure,42.525050,3.083155,282.741429,0.335714
1,Studette idéalement située - Collioure,"42.52393500,3.08446940",None,"Situé à Collioure, à seulement 500 mètres du c...",26,42.523935,3.084469,26,Collioure,42.525050,3.083155,282.741429,0.335714
2,T3 La Crique- Piscine -Terrasses -Wifi -Parking,"42.52713180,3.07614980",None,"Situé à Collioure, à seulement 1,6 km du châte...",26,42.527132,3.076150,26,Collioure,42.525050,3.083155,282.741429,0.335714
3,Coconut - appartement T2 à Collioure,"42.52215520,3.08856820","8,0","Situated in Collioure, 300 metres from Boutigu...",26,42.522155,3.088568,26,Collioure,42.525050,3.083155,282.741429,0.335714
4,Hôtel Princes de Catalogne,"42.52591431,3.08268160","7,9",Installé dans la ville pittoresque de Colliour...,26,42.525914,3.082682,26,Collioure,42.525050,3.083155,282.741429,0.335714
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Vieux Port République Cannebiére,"43.29594971,5.37455259","8,7",Vous pouvez bénéficier d'une réduction Genius ...,19,43.295950,5.374553,19,Marseille,43.296174,5.369953,281.592857,0.234286
96,"Holiday Inn Express Marseille Saint Charles, a...","43.30272424,5.37933044","7,8",L'Holiday Inn Express Marseille Saint Charles ...,19,43.302724,5.379330,19,Marseille,43.296174,5.369953,281.592857,0.234286
97,Marseillement votre,"43.30020105,5.38260245","8,3",Vous pouvez bénéficier d'une réduction Genius ...,19,43.300201,5.382602,19,Marseille,43.296174,5.369953,281.592857,0.234286
98,LE PANIER ENCHANTÉ - Suites & Loft - Terrasse ...,"43.30009700,5.36758500","7,9",Vous pouvez bénéficier d'une réduction Genius ...,19,43.300097,5.367585,19,Marseille,43.296174,5.369953,281.592857,0.234286


# Création d'une carte pour le top 5 des 20 meilleurs hotels


In [ ]:
fig = px.scatter_mapbox(
        hotels, 
        lat="lat_x", 
        lon="long_x",
        mapbox_style="carto-positron",
        zoom=5,
        hover_name='name',
    color='city'
)
fig.update_traces(marker={'size' : 5})

fig.show()

# Store to s3

Save all the results to a S3 bucket

In [ ]:
import boto3

In [ ]:
AWS_ACCESS_KEY="YOUR_KEY"
AWS_SECRET_ACCESS_KEY="YOUR_SECRET_KEY"

In [ ]:
session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3 = session.client("s3")

bucket = s3.create_bucket(Bucket="kayak-project",
                 CreateBucketConfiguration={'LocationConstraint' : 'eu-west-3'})

In [ ]:
s3.put_object(
            Body=cities.to_csv(index=False),
            Bucket='kayak-project',
            Key='hotel.json',
)

In [ ]:
hotels = pd.read_json('results/hotels_top5.json')

s3.put_object(
            Body=hotels.to_csv(index=False),
            Bucket='kayak-project',
            Key='Hotel.json',
)

# Load to RDS

Récupération les données du compartiment S3 et ajoutez-les à une base de données Postgresql à l'aide de Sqlachemy

pas pu le faire car problème d'installation de postgress sur l'ordinateur